In [115]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog

from joblib import dump, load

In [116]:
signatures_path = os.path.join(str(globals()['_dh'][0]).replace("notebooks", ""), "signatures")
person_folders = sorted([d for d in os.listdir(signatures_path) if os.path.isdir(os.path.join(signatures_path, d))], key = lambda x: int(x.split("_")[1]))

In [117]:
def pre_process(img_path):
  
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img_resized = cv2.resize(img, (500, 300))
  img_normalized = img_resized / 255

  return img_normalized

In [119]:
def extract_features(img):

    hog_features = hog(img, orientations = 8, pixels_per_cell = (16,16), cells_per_block = (1,1), feature_vector = True)
    
    return hog_features

In [129]:
# prepare the data to train the signature recognition SVM: for this model only the original signatures will be used
def prepare_recognition_data():

    # arrays to store the features and the personal labels
    features = []
    y_data = []

    for person_folder in person_folders:
        # array to store the features of all signatures of a person
        individual_features = []
        # iterate over all signatures of a person
        for signature_file in sorted(os.listdir(os.path.join(signatures_path, person_folder)), key = lambda x: int(x.split(".")[0].split("_")[2])):
            if signature_file.startswith("original") and signature_file.endswith(".png"):
                image_path = os.path.join(signatures_path, person_folder, signature_file)
                preprocessed_image = pre_process(image_path)
                # concatenate all images's features into one array
                individual_features = np.concatenate((individual_features, extract_features(preprocessed_image)))
        
        features.append(individual_features)
        y_data.append(int(person_folder.split("_")[1]))

    X_data = [[float(feature) for feature in feature_vec] for feature_vec in features]
    X = np.array(X_data)
    y = np.array(y_data)

    return X, y

In [130]:
X, y = prepare_recognition_data()

In [131]:
print(X)
print(X.shape)
print(y)
print(y.shape)

[[0.36079837 0.36079837 0.36079837 ... 0.36867658 0.36867658 0.22032487]
 [0.38543391 0.33612732 0.38543391 ... 0.25476781 0.37716204 0.37716204]
 [0.38183283 0.38183283 0.38183283 ... 0.33019551 0.35676541 0.35676541]
 ...
 [0.35355339 0.35355339 0.35355339 ... 0.36366662 0.36366662 0.36366662]
 [0.35514234 0.35514234 0.35514234 ... 0.36914348 0.36914348 0.36914348]
 [0.35355339 0.35355339 0.35355339 ... 0.36067429 0.36067429 0.29899564]]
(55, 107136)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55]
(55,)


In [132]:
# SVM FOR SIGNATURE RECOGNITION

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(38, 107136)
(17, 107136)
(38,)
(17,)


In [133]:
svm_recognition = SVC()
svm_recognition.fit(X_train, y_train)

SVC()

In [135]:

y_pred = svm_recognition.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
dump(svm_recognition, "../models/svm_recognition.joblib")


              precision    recall  f1-score   support

           4       0.00      0.00      0.00       1.0
           6       0.00      0.00      0.00       1.0
           7       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       1.0
          13       0.00      0.00      0.00       1.0
          14       0.00      0.00      0.00       1.0
          16       0.00      0.00      0.00       0.0
          18       0.00      0.00      0.00       1.0
          20       0.00      0.00      0.00       1.0
          27       0.00      0.00      0.00       1.0
          32       0.00      0.00      0.00       1.0
          33       0.00      0.00      0.00       1.0
          34       0.00      0.00      0.00       1.0
          35       0.00      0.00      0.00       1.0
          42       0.00      0.00      0.00       1.0
          44       0.00      0.00      0.00       1.0
          50       0.00      0.00      0.00       1.0
          52       0.00    

/home/matteo/anaconda3/envs/signature-forgery-detection/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matteo/anaconda3/envs/signature-forgery-detection/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/matteo/anaconda3/envs/signature-forgery-detection/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this be

['../models/svm_recognition.joblib']

In [ ]:
# SVM FOR SIGNATURE VERIFICATION

# TO-DO
